In [2]:
import torch


In [90]:
label_k400_train = '/data/jong980812/project/Video-CBM/data/video_annotation/kinetics400/train.csv'
import pandas as pd
cleaned = pd.read_csv(label_k400_train, header=None, delimiter=',')
label_array = list(cleaned.values[:, 1])
label = torch.tensor(label_array,dtype=torch.int32)
internvid_image_feat = '/data/jong980812/project/Video-CBM/results/k400/class_names/feature_directly/activation/kinetics400_train_clip_ViT-B16.pt'
internvid_text_feat = '/data/jong980812/project/Video-CBM/results/k400/class_names/feature_directly/activation/kinetics400_classes_ViT-B16.pt'
internvid_image_feat = torch.load(internvid_image_feat)
internvid_text_feat = torch.load(internvid_text_feat)


In [96]:
label_k400_val = '/data/jong980812/project/Video-CBM/data/video_annotation/kinetics400/val.csv'
import pandas as pd
cleaned = pd.read_csv(label_k400_val, header=None, delimiter=',')
label_array = list(cleaned.values[:, 1])
label = torch.tensor(label_array,dtype=torch.int32)
internvid_image_feat = '/data/jong980812/project/Video-CBM/results/k400/class_names/feature_directly/activation/kinetics400_val_clip_ViT-B16.pt'
internvid_text_feat = '/data/jong980812/project/Video-CBM/results/k400/class_names/feature_directly/activation/kinetics400_classes_ViT-B16.pt'
internvid_image_feat = torch.load(internvid_image_feat)
internvid_text_feat = torch.load(internvid_text_feat)


In [79]:
label_ucf101_train = '/data/jong980812/project/Video-CBM/data/video_annotation/UCF101/train.csv'
import pandas as pd
cleaned = pd.read_csv(label_ucf101_train, header=None, delimiter=' ')
ucf101_train_label_array = list(cleaned.values[:, 1])
internvid_image_feat = '/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/UCF101_train_clip_ViT-B16.pt'
internvid_text_feat = '/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/ucf101_classes_ViT-B16.pt'
internvid_image_feat = torch.load(internvid_image_feat)
internvid_text_feat = torch.load(internvid_text_feat)
label ='/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/label_train.txt'
with open(label,'r',encoding='utf-8') as file:
    lines = file.readlines()
lines = [int(line.strip()) for line in lines]
label = torch.tensor(lines)



# #!!!
# label_ucf101_val = '/data/jong980812/project/Video-CBM/data/video_annotation/UCF101/val.csv'
# import pandas as pd
# cleaned = pd.read_csv(label_ucf101_val, header=None, delimiter=' ')
# ucf101_val_label_array = list(cleaned.values[:, 1])
# internvid_image_feat = '/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/UCF101_val_clip_ViT-B16.pt'
# internvid_text_feat = '/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/ucf101_classes_ViT-B16.pt'
# internvid_image_feat = torch.load(internvid_image_feat)
# internvid_text_feat = torch.load(internvid_text_feat)
# label ='/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/label_val.txt'
# with open(label,'r',encoding='utf-8') as file:
#     lines = file.readlines()
# lines = [int(line.strip()) for line in lines]
# label = torch.tensor(lines)


In [78]:
internvid_image_feat = '/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/UCF101_val_clip_ViT-B16.pt'
internvid_text_feat = '/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/ucf101_classes_ViT-B16.pt'
label ='/data/jong980812/project/Video-CBM/results/ucf101/class_list/activation/label_train.txt'
internvid_image_feat = torch.load(internvid_image_feat)
internvid_text_feat = torch.load(internvid_text_feat)
# label= torch.load(label)
with open(label,'r',encoding='utf-8') as file:
    lines = file.readlines()
lines = [int(line.strip()) for line in lines]

In [49]:
label = torch.tensor(label_k400_train,dtype=torch.int32)
print(label[-1])

tensor(24, dtype=torch.int32)


In [92]:
internvid_image_feat.shape,internvid_text_feat.shape,label.shape

(torch.Size([240436, 512]), torch.Size([400, 512]), torch.Size([240436]))

In [89]:
def cosine_similarity(vec1, vec2):
    dot_product = torch.matmul(vec1, vec2.T)  # 벡터의 내적
    vec1_norm = vec1.norm(dim=1, keepdim=True)  # 이미지 벡터의 크기 계산
    vec2_norm = vec2.norm(dim=1, keepdim=True)  # 텍스트 벡터의 크기 계산
    return dot_product / (vec1_norm * vec2_norm.T)

# 각 이미지에 대해 가장 유사한 텍스트의 인덱스를 찾는 함수
def find_most_similar_text(image_feats, text_feats):
    similarities = cosine_similarity(image_feats, text_feats)  # 모든 유사도 계산
    most_similar_indices = torch.argmax(similarities, dim=1)    # 가장 큰 유사도의 텍스트 인덱스 찾기
    return most_similar_indices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 가장 유사한 텍스트 인덱스 저장
print(device)


cuda


In [73]:
def find_top3_similar_text(image_feats, text_feats):
    similarities = cosine_similarity(image_feats, text_feats)  # 모든 유사도 계산
    top3_similarities, top3_indices = torch.topk(similarities, k=10, dim=1)  # 상위 3개의 유사도와 인덱스 찾기
    return top3_indices

# accuracy를 계산하는 함수
def compute_accuracy_top3(image_labels, top3_indices):
    correct = 0
    for i in range(image_labels.size(0)):
        if image_labels[i] in top3_indices[i]:  # 라벨이 top3 인덱스에 있는지 확인
            correct += 1
    accuracy = correct / image_labels.size(0)
    print(f'Number of correct elements: {correct}')
    print(f'Accuracy: {accuracy * 100:.2f}%')
    return accuracy

In [97]:
most_similar_text_indices = find_most_similar_text(internvid_image_feat.to(device), internvid_text_feat.to(device))
top3_indices = find_top3_similar_text(internvid_image_feat.to(device), internvid_text_feat.to(device))


In [98]:
most_similar_text_indices=most_similar_text_indices.cpu()
correct = (most_similar_text_indices == label)

# 맞는 원소의 개수를 구함
num_correct = correct.sum().item()

# accuracy 계산 (전체 개수 중에서 몇 퍼센트가 맞았는지)
accuracy = num_correct / most_similar_text_indices.size(0)

print(f'Number of correct elements: {num_correct}')
print(f'Accuracy: {accuracy * 100:.2f}%')

Number of correct elements: 9618
Accuracy: 48.59%


In [99]:
accuracy = compute_accuracy_top3(label, top3_indices)

Number of correct elements: 16131
Accuracy: 81.49%


In [40]:
print(accuracy)

0.6723075368761366


In [34]:
most_similar_text_indices.max()

tensor(100)